<a href="https://colab.research.google.com/github/lhaq/llm-eval/blob/main/llmEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#! pip install python-dotenv --quiet

In [24]:
#! pip install groq deepeval openai --quiet

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [ ]:
from google.colab import userdata
GROK_API_KEY = userdata.get('GROK_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
TURSO_AUTH_TOKEN = userdata.get('TURSO_AUTH_TOKEN')

In [ ]:
import os

from groq import Groq
from openai import OpenAI
from dotenv import load_dotenv
from google.colab import userdata

load_dotenv()

user_content = "Give a brief history about Martin Luther King, ensure it is only a short paragraph"
model_name = "llama3-8b-8192"

client = Groq(
    # This is the default and can be omitted
    api_key=userdata.get("GROK_API_KEY"),
)

chat_completion_llama = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful customer assistant in for a company that sells cars. Do not answer any question that are outside the remit of a customer assistant of a car sales company, only keep it to your function"
        },
        {
            "role": "user",
            "content": f"{user_content}",
        }
    ],
    model=f"{model_name}",
    #response_format={ "type": "json_object" },

)

# Assign the reponse from the LLM to a variable
response = chat_completion_llama.choices[0].message.content
print(response)

client2 = Groq(
    # This is the default and can be omitted
    api_key=userdata.get("GROK_API_KEY"),
)



chat_completion_gemma = client2.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful customer assistant in for a company that sells cars. Do not answer any question that are outside the remit of a customer assistant of a car sales company, if asked always prompt the user to ask something relevant"},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ],
    model="gemma2-9b-it",

)

response2 = chat_completion_gemma.choices[0].message.content
print(response2)

client3 = Groq(
    # This is the default and can be omitted
    api_key=userdata.get("GROK_API_KEY"),
)

chat_completion_mixtral = client3.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful customer assistant in for a company that sells cars. Do not answer any question that are outside the remit of a customer assistant of a car sales company, if asked always prompt the user to ask something relevant"},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ],
    model="mixtral-8x7b-32768",
)

response3 = chat_completion_mixtral.choices[0].message.content
print(response3)







I'm happy to help with a question, but I must politely decline as it is outside my remit as a customer assistant for a car sales company. My role is to assist with car-related inquiries, not historical figures. If you'd like to learn more about our latest car models, I'd be more than happy to help!
Function calls itself,
Unfurling like a flower bright,
Till base case takes root. 

Let me know if you have any questions about our vehicles or financing options. 😄 

Endless function calls,
Recursion reveals solution,
Code's artful dance.


Create Web App

In [33]:
from logging import PlaceHolder
# Install necessary packages if not already installed
!pip install gradio -q
!pip install openai -q
!pip install python-dotenv -q
!pip install deepeval -q

import gradio as gr
import os
from groq import Groq
from dotenv import load_dotenv
from google.colab import userdata
from deepeval.metrics import FaithfulnessMetric, AnswerRelevancyMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase

load_dotenv()

# Function to process the query based on model selection
def get_response(model_name, query):
    client = Groq(api_key=userdata.get("GROK_API_KEY"))

    # Define the system message based on the model
    system_message = {
        "role": "system",
        "content": "You are a helpful customer assistant in for a company that sells cars. Do not answer any question that are outside the remit of a customer assistant of a car sales company, only keep it to your function"
    }

    # Create the chat completion based on the selected model
    chat_completion = client.chat.completions.create(
        messages=[
            system_message,
            {
                "role": "user",
                "content": query,
            }
        ],
        model=model_name,
    )

    # Assign the response from the LLM to a variable
    response = chat_completion.choices[0].message.content
    return response

# Function to evaluate the response using deepeval
def evaluate_response(response):
    api_key = userdata.get("OPENAI_API_KEY")
    test_case = LLMTestCase(
        input=response,
        actual_output="I'm here to assist you with questions specifically about our Cars Sales company's products and services. If you have any related inquiries related to that, please let me know!",
        retrieval_context=["You are a helpful customer assistant in for a company that sells cars. Do not answer any question that are outside the remit of a customer assistant of a car sales company, only keep it to your function"]
    )

    metrics = {
        "Faithfulness": FaithfulnessMetric(threshold=0.5),
        "Answer Relevancy": AnswerRelevancyMetric(threshold=0.5),
        "Contextual Relevancy": ContextualRelevancyMetric(threshold=0.5)
    }

    results = []
    for name, metric in metrics.items():
        metric.measure(test_case)
        results.append(f"{name}:\n  Score: {metric.score}\n  Reason: {metric.reason}\n  Is Successful: {metric.is_successful()}\n")

    return "\n".join(results)

# Create the Gradio interface

theme=gr.themes.Soft()


with gr.Blocks(theme=theme) as llm_evaluation_app:
    gr.Markdown("# LLM Evaluation")

    # Dropdown for model selection
    model_name = gr.Dropdown(
        ["select model here", "llama3-8b-8192", "gemma2-9b-it", "mixtral-8x7b-32768"],
        label="Model Name",
        value="select model here"
    )


    # Textbox for long queries
    query = gr.Textbox(
        placeholder="Enter your long query here...",
        label="Query"
    )

    # Button to trigger processing
    submit_button = gr.Button("Submit")

    # Output box for response
    response = gr.Textbox(
        label="Response",
        interactive=False
    )

    # Output box for evaluation metrics
    evaluation_metrics = gr.Textbox(
        label="Evaluation Metrics",
        interactive=False
    )

    # Connect components with the processing function
    submit_button.click(get_response, inputs=[model_name, query], outputs=response)
    submit_button.click(evaluate_response, inputs=response, outputs=evaluation_metrics)

# Launch the app
llm_evaluation_app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1140d1da9adefeb345.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
